<a href="https://colab.research.google.com/github/bryandaetz1/CA_brewery_map/blob/main/ca_brewery_map_altair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [187]:
!pip install geopandas

In [188]:
import pandas as pd
import altair as alt
import geopandas as gpd

In [189]:
ca_map = gpd.read_file('https://raw.githubusercontent.com/ropensci/geojsonio/master/inst/examples/california.geojson')
beer_df = pd.read_csv('https://raw.githubusercontent.com/bryandaetz1/CA_brewery_map/main/beers.csv')
brew_df = pd.read_csv('https://raw.githubusercontent.com/bryandaetz1/CA_brewery_map/main/breweries.csv')

In [190]:
brew_viz = brew_df[brew_df['latitude'] != 0]

In [191]:
brew_viz.head()

,Brewery,Address,Contact_Info,latitude,longitude
0,Alameda Island Brewing Company,"1716 Park Street\nAlameda, California, United ...",http://alamedaislandbrewingcompany.com/,37.768393,-122.238778
1,Faction Brewing,"2501 Monarch Street\nAlameda, California, Unit...",http://factionbrewing.com/,37.786495,-122.309290
2,Ocean View Brew Works,"627 San Pablo Avenue\nAlbany, California, Unit...",https://www.oceanviewbrews.com/,37.894208,-122.299982
3,Ohana Brewing Company,"7 S 1st Street\nAlhambra, California, United S...",http://ohanabrew.com,34.094288,-118.127880
4,Alpine Beer Company,"2363 Alpine Blvd\nAlpine, California, United S...",http://alpinebeerco.com/home/,32.835185,-116.762648


In [192]:
brew_viz['Beers'] = ''
brew_viz['Styles'] = ''
brew_viz['Number of Beers'] = 0

for i, brew in enumerate(brew_viz['Brewery']):
  
  #storing list of beers in Beers column
  beer_list = beer_df[beer_df['Brewery'] == brew]['Beer'].to_list()
  brew_viz['Beers'][i] = beer_list
  
  #storing # of beers in num_beers column
  num_beers = 0
  for a in beer_list:
    num_beers = num_beers + 1
  brew_viz['Number of Beers'][i] = num_beers

  #storing list of styles in Styles column
  style_list = beer_df[beer_df['Brewery'] == brew]['Style'].to_list()
  brew_viz['Styles'][i] = style_list

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [193]:
brew_viz.head()

,Brewery,Address,Contact_Info,latitude,longitude,Beers,Styles,Number of Beers
0,Alameda Island Brewing Company,"1716 Park Street\nAlameda, California, United ...",http://alamedaislandbrewingcompany.com/,37.768393,-122.238778,"[Alameda’s IPA, Autumnator Doppelbock, Ballena...","[American IPA, Doppelbock, Milk / Sweet Stout,...",18
1,Faction Brewing,"2501 Monarch Street\nAlameda, California, Unit...",http://factionbrewing.com/,37.786495,-122.309290,"[A-Town Pale, Diverge IPA, Faction Pils, Facti...","[American Pale Ale (APA), American IPA, German...",7
2,Ocean View Brew Works,"627 San Pablo Avenue\nAlbany, California, Unit...",https://www.oceanviewbrews.com/,37.894208,-122.299982,[],[],0
3,Ohana Brewing Company,"7 S 1st Street\nAlhambra, California, United S...",http://ohanabrew.com,34.094288,-118.127880,[Spa Water Saison],[Saison / Farmhouse Ale],1
4,Alpine Beer Company,"2363 Alpine Blvd\nAlpine, California, United S...",http://alpinebeerco.com/home/,32.835185,-116.762648,"[Alpine Ale, Captain Stout, Duet, Irish Red, M...","[American Pale Ale (APA), Oatmeal Stout, Ameri...",7


In [194]:
# extracting city from address and saving it as a new column
brew_viz['City'] = brew_viz['Address'].apply(lambda x: x.split(', California')[0].split('\n')[-1])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [195]:
#creating background
background = alt.Chart(ca_map).mark_geoshape(
    fill='#3d3d3b',
    stroke='black'
).properties(
    width = 500,
    height = 700
)

brush = alt.selection(type='multi')

#plotting breweries
points = alt.Chart(brew_viz).mark_circle().encode(
    longitude = 'longitude:Q',
    latitude = 'latitude:Q',
    color = alt.value('#e3b434'),
    size = alt.Size('Number of Beers:Q', legend = None),
    tooltip = ['Brewery:N',
               'City:N']
).properties(
    title = 'California Brewery Map'
).add_selection(brush)

#base chart for beer data tables
ranked_text = alt.Chart(brew_viz).mark_text().encode(
    y = alt.Y('row_number:O', axis = None)
).transform_window(
    row_number='row_number()'
).transform_filter(
    brush
).transform_window(
    rank='rank(row_number)'
).transform_filter(
    alt.datum.rank <= 15
)

#beer data tables to display
brewery = ranked_text.encode(text='Brewery:N').properties(title='Brewery')
beer = ranked_text.encode(text='Address:N').properties(title='Address')
style = ranked_text.encode(text='Contact_Info:N').properties(title='Contact Info')
text = alt.hconcat(brewery, beer, style)

#bar chart visualizing number of beers
bar = alt.Chart(brew_viz).mark_bar().encode(
    y = 'Brewery:N',
    x = 'Number of Beers:Q',
    color = alt.value('#e3b434'),
    tooltip = 'Beers'
).transform_filter(
    brush
).transform_window(
    rank = 'rank(Number of Beers)',
    sort = [alt.SortField('Number of Beers', order = 'descending')]
).transform_filter(
    alt.datum.rank <= 15
)    

# Build chart
background + points | alt.vconcat(text, bar)

alt.HConcatChart(...)